<a href="https://colab.research.google.com/github/Rob192/verbatim/blob/master/trial_lda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Rob192/verbatim/master/COVID-19-RETEX_5-25-2020_pavel.csv', sep = ',')
df.head()

,N°Obs,1. Quel est votre ministère d'appartenance ?,2. Quelle est votre fonction ?,3. Précisez votre fonction,"4. Quel est le nom de votre direction, sous-direction ou service",5. Quelle est votre région (pour les administrations territorial,6. A1. Considérez-vous avoir eu rapidement les moyens de mobilis,7. A1a. Pour quelles raisons ?,8. A2. Dans quel délai avez-vous eu connaissance des moyens numé,9. A3. Comment notez-vous l'efficacité de vos moyens de télétrav,10. A4. Quelles initiatives complémentaires avez-vous prises pou,11. Messagerie du ministère,12. Messagerie autre (ex: gmail),13. Messagerie instantanée interministérielle (Tchap),14. Messagerie instantanée autre (ex: WhatsApp),15. Visio-conférence du ministère,16. Viso-conférence interministériel (WebConf),17. Visio-conférence autre (ex: Skype),18. Outils collaboratifs du ministère,"19. Outils collaboratifs interministériels (OSMOSE, PLANO, RESAN",20. Outils collaboratifs autres,21. Audio-conférence téléphonique,22. B2. Avez-vous eu recours à des ressources ou compétences add,"23. B2a. Si oui, lesquelles et comment les avez-vous activées ?",24. B3. A quelles principales difficultés avez-vous fait face po,25. C1. Considérez-vous que votre direction disposait d'une cult,26. C2. Quels moyens de communication internes avez-vous mis en,"27. C2a. Si ""autre"", lesquelles ?","28. C3. Avez-vous détecté des agents mis en difficulté, y compri",29. C3a. Que pouvez-vous en dire ?,30. C4. Entendez-vous conserver au-delà de la crise des pratique,"31. C4a. Si oui, lesquelles ?",32. C5. Que souhaitez-vous ajouter sur les impacts du numérique,33. Clé,34. Date de saisie,35. Date de dernière modification,36. Date de dernier enregistrement,37. Temps de saisie,38. Langue,39. Progression,40. Origine,41. Appareil utilisé pour la saisie
0,1,Services du Premier ministre,Autre,secrétaire générale,NaN,NaN,Oui,NaN,Dès la 1ère semaine,4,NaN,Très utile,Pas utile,Non utilisé,Non utilisé,Non utilisé,Non utilisé,Utile,Non utilisé,Non utilisé,Utile,Très utile,Non,NaN,insuffisance de la bande passante; lenteurs;,Oui mais insuffisamment,Courriels / Lettre d'information;Enquête en li...,NaN,Oui,handicap; isolement;,Oui,fluidité des validations électroniques; visio ...,NaN,66DP-LXBD,28/04/2020 15:01:25,NaN,28/04/2020 15:05:50,264.940515,FR,Terminé,Direct,PC
1,2,Services du Premier ministre,Directeur.trice (centrale),NaN,CIVEN,NaN,Pas suffisamment,Manque de postes PC portables avec accès télét...,Dès la 1ère semaine,5,Ensemble du travail du CIVEN se fait par télét...,Très utile,Utile,Non utilisé,Utile,Non utilisé,Non utilisé,Utile,Non utilisé,Non utilisé,Utile,Très utile,Oui,Travail sur l'application informatique du CIVE...,manque de postes informatiques télétravail pou...,Oui mais insuffisamment,Courriels / Lettre d'information;Réseau social...,NaN,Non,NaN,Oui,Recours au télétravail et mise place d'audioco...,"La période de confinement, malgré le soutien d...",TNTW-XFV4,28/04/2020 15:06:54,NaN,28/04/2020 15:18:24,690.338652,FR,Terminé,Direct,PC
2,3,Services du Premier ministre,Autre,Secrétaire générale,Délégation interministérielle aux jeux Olympiq...,NaN,Oui,NaN,Dès la 1ère semaine,5,Maintien de réunions de service hebdomadaires ...,Très utile,Pas utile,Inconnu,Très utile,Non utilisé,Non utilisé,Très utile,Non utilisé,Inconnu,Non utilisé,Très utile,Non,NaN,Aucune difficulté,Oui,Courriels / Lettre d'information;Réseau social...,NaN,Non,NaN,Je ne sais pas encore,NaN,Néant,AK5M-Y4WQ,28/04/2020 15:11:49,NaN,28/04/2020 15:24:11,742.120496,FR,Terminé,Direct,PC
3,4,Services du Premier ministre,Autre,Secrétaire général,Ecole nationale d'administration,NaN,Oui,NaN,Jamais,5,"En tant qu'établissement public administratif,...",Non utilisé,Non utilisé,Non utilisé,Non utilisé,Non utilisé,Non utilisé,Très utile,Non utilisé,Non utilisé,Très utile,Très utile,Non,NaN,"Les outils ne sont pas en cause, comme indiqué...",Oui,Courriels / Lettre d'information;Site Web;Web ...,Le dialogue social. J'ai réuni les organisatio...,

In [0]:
cols = df.columns

In [0]:
df = df['31. C4a. Si oui, lesquelles ?']
df.columns = ["bonnes_pratiques"]

In [5]:
df.head()

0    fluidité des validations électroniques; visio ...
1    Recours au télétravail et mise place d'audioco...
2                                                  NaN
3    Il est prévu dans le PRAP une période de maint...
4                                                  NaN
Name: 31. C4a. Si oui, lesquelles ?, dtype: object

In [0]:
df.dropna(inplace=True)

In [0]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.fr.stop_words import STOP_WORDS

from tqdm import tqdm_notebook as tqdm
from pprint import pprint

In [12]:
import spacy.cli
spacy.cli.download("fr_core_news_md")

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [0]:
nlp = spacy.load('fr_core_news_md')

In [0]:
# My list of stop words.
stop_list = ["\n "]

# Updates spaCy's default stop words list with my additional words. 
nlp.Defaults.stop_words.update(stop_list)

# Iterates over the words in the stop words list and resets the "is_stop" flag.
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

In [15]:
txt = df[1]
doc = nlp(txt)
txt

"Recours au télétravail et mise place d'audioconférence pour certaines réunions."

In [16]:
doc

Recours au télétravail et mise place d'audioconférence pour certaines réunions.

In [17]:
def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
lemmatizer(doc)

recours au télétravail et mise place de audioconférence pour certain réunion .

In [18]:
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc
remove_stopwords(doc)

['Recours', 'télétravail', 'mise', 'place', 'audioconférence', 'réunions']

In [0]:
nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
nlp.add_pipe(remove_stopwords, name="stopwords", last=True)

In [20]:
doc_list = []
# Iterates through each article in the corpus.
for txt in tqdm(df.values):
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(txt)
    doc_list.append(pr)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
# Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(doc_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in doc_list]

In [0]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=3, 
                                           random_state=2,
                                           update_every=1,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [23]:
pprint(lda_model.print_topics(num_words=10))

[(0,
  '0.041*"télétravail" + 0.019*"réunion" + 0.018*"outil" + 0.012*"agent" + '
  '0.011*"travail" + 0.010*"collaboratif" + 0.009*"distance" + '
  '0.008*"important" + 0.008*"utilisation" + 0.008*"recours"'),
 (1,
  '0.043*"télétravail" + 0.036*"visio" + 0.024*"audio" + 0.019*"conférence" + '
  '0.018*"réunion" + 0.014*"agent" + 0.013*"travail" + 0.013*"outil" + '
  '0.010*"pratique" + 0.010*"utilisation"'),
 (2,
  '0.072*"télétravail" + 0.018*"agent" + 0.017*"jour" + 0.015*"travail" + '
  '0.014*"  " + 0.012*"temps" + 0.012*"déplacement" + 0.012*"distance" + '
  '0.012*"semaine" + 0.010*"pouvoir"')]


In [0]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=10, 
                                           random_state=2,
                                           update_every=1,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [25]:
pprint(lda_model.print_topics(num_words=10))

[(0,
  '0.037*"télétravail" + 0.022*"réunion" + 0.019*"agent" + 0.012*"permettre" + '
  '0.011*"faire" + 0.010*"équipe" + 0.009*"pouvoir" + 0.009*"fréquent" + '
  '0.009*"service" + 0.008*"usage"'),
 (1,
  '0.024*"agent" + 0.022*"télétravail" + 0.021*"réunion" + 0.021*"audio" + '
  '0.020*"visio" + 0.015*"travail" + 0.015*"pratique" + 0.012*"utilisation" + '
  '0.012*"pouvoir" + 0.011*"outil"'),
 (2,
  '0.044*"  " + 0.042*"télétravail" + 0.016*"travail" + 0.015*"jour" + '
  '0.013*"faire" + 0.013*"semaine" + 0.011*"temps" + 0.011*"pouvoir" + '
  '0.011*"crise" + 0.008*"probablement"'),
 (3,
  '0.054*"télétravail" + 0.024*"visio" + 0.023*"agent" + 0.015*"jour" + '
  '0.014*"conférence" + 0.014*"travail" + 0.012*"pouvoir" + 0.012*"extension" '
  '+ 0.012*"distance" + 0.011*"équipe"'),
 (4,
  '0.032*"télétravail" + 0.018*"réunion" + 0.011*"  " + 0.011*"accès" + '
  '0.009*"pouvoir" + 0.008*"tâche" + 0.008*"visioconférence" + '
  '0.008*"renforcement" + 0.008*"pratique" + 0.007*"place"'),
